In [ ]:
#!pip install xgboost

In [ ]:
import os
import pandas as pd
import numpy as np

import psycopg2
import sqlalchemy 
from sqlalchemy import create_engine
from sqlalchemy import text

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

from datetime import timedelta

In [ ]:
with open("credentials/bbdd.json") as f:
    bbdd = json.load(f)

host=bbdd["host"]
database=bbdd["database"]
user=bbdd["user"]
password=bbdd["password"]

In [ ]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{database}")

In [ ]:
#Bloque de test - cierra automáticamente la conexión
with engine.connect() as conn:
    rows = conn.execute(text("SELECT 1")).all()
    df = pd.read_sql_table(table_name = "demanda_electrica", con = conn)
    print(df.head())

In [ ]:
connection = engine.connect()

In [ ]:
df = pd.read_sql_table(table_name = "demanda_electrica", con = connection)

In [ ]:
df

In [ ]:
connection.close()

In [ ]:
engine.dispose()

In [22]:
df_backup = df.copy()

#df = df_backup

In [18]:
df_clean = df.copy()

,value,datetime,datetime_utc,tz_time,geo_id,geo_name
0,3187298.0,2014-01-01T00:00:00.000+01:00,2013-12-31T23:00:00Z,2013-12-31T23:00:00.000Z,8741,Península
1,3916726.0,2014-01-02T00:00:00.000+01:00,2014-01-01T23:00:00Z,2014-01-01T23:00:00.000Z,8741,Península
2,3957472.0,2014-01-03T00:00:00.000+01:00,2014-01-02T23:00:00Z,2014-01-02T23:00:00.000Z,8741,Península
3,3726719.0,2014-01-04T00:00:00.000+01:00,2014-01-03T23:00:00Z,2014-01-03T23:00:00.000Z,8741,Península
4,3511386.0,2014-01-05T00:00:00.000+01:00,2014-01-04T23:00:00Z,2014-01-04T23:00:00.000Z,8741,Península


In [31]:
df_clean.isnull().sum()

value           0
datetime        0
datetime_utc    0
tz_time         0
geo_id          0
geo_name        0
dtype: int64

In [33]:
df_clean["geo_name"].value_counts(dropna=False)

geo_name
Península    4276
Name: count, dtype: int64

In [30]:
df_clean["geo_id"].value_counts(dropna=False)

geo_id
8741    4276
Name: count, dtype: int64